# Лабораторная работа 1 (KNN)

## Пояснения к выбору начальных условий

### Регрессия

#### Датасет:
Для решения задачи регрессии был выбран датасет "Salary data" ([link](https://www.kaggle.com/datasets/rkiattisak/salaly-prediction-for-beginer)). 

**Практическое применение**: Результаты могут помочь компаниям лучше понимать, как различные факторы влияют на зарплату, или использовать модель для автоматизации назначения зарплаты новым сотрудникам.

#### Метрики:

Для оценки модели были выбраны следующие метрики:

1. Mean Absolute Error (MAE) - Среднее абсолютное отклонение
2. Mean Squared Error (MSE) - Среднеквадратичное отклонение
3. R-squared ($R^2$) – Коэффициент детерминации

### Классификация

#### Датасет:

Для решения задачи классификации был выбран датасет "Student Depression Dataset" ([link](https://www.kaggle.com/datasets/hopesb/student-depression-dataset)). 

**Практическое применение**: Модели машинного обучения, обученные на этом датасете, могут помочь в раннем выявлении депрессии у студентов, анализируя взаимосвязь между факторами, такими как академическое давление, финансовый стресс и привычки сна. Это позволяет образовательным учреждениям и организациям разрабатывать целенаправленные меры поддержки для улучшения ментального здоровья учащихся.

#### Метрики:

Для оценки модели были выбраны следующие метрики:

1. Accuracy – Точность
2. Precision – Точность предсказания для одного класса
3. Recall – Полнота
4. F1-score – Среднее гармоническое между Precision и Recall

#### Импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table

### Regression

#### 1. Обработка данных

In [2]:
df = pd.read_csv('data/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5.0,90000
1,28,Female,Master's,Data Analyst,3.0,65000
2,45,Male,PhD,Senior Manager,15.0,150000
3,36,Female,Bachelor's,Sales Associate,7.0,60000
4,52,Male,Master's,Director,20.0,200000


In [3]:
df['Job Title'].nunique()

174

В признаке `Job Title` слишком много классов, поэтому лучше избавится от него 

In [4]:
df = df.drop(columns=['Job Title'])

In [5]:
X, y = df.drop(columns=['Salary']), df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# X_train, y_train = X, y

gender_le = LabelEncoder()
el_le = LabelEncoder()

X_train['Gender'] = gender_le.fit_transform(X_train['Gender'])
X_test['Gender'] = gender_le.transform(X_test['Gender'])

X_train['Education Level'] = el_le.fit_transform(X_train['Education Level'])
X_test['Education Level'] = el_le.transform(X_test['Education Level'])

#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

Построим модель используя реализацию из sklearn

In [6]:
metrics = regression_cross_validate(KNeighborsRegressor, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_table(*metrics)

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      | 11422.6       | 812.697       |
| MSE      |     3.029e+08 |   3.00217e+07 |
| R2       |     0.869525  |   0.0147744   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 259533832.74
Средняя абсолютная ошибка (MAE): 11612.23
Коэффициент детерминации (R^2): 0.88


Как можем увидеть значение метрики $R^2$ около 0.88, что означает что около 88% дисперсии данных объясняется моделью. Данное значение является очень хорошим, для такой простой модели.

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить число соседей

In [ ]:
onehot = OneHotEncoder(sparse_output=False, drop='first')

categorical_features = ['Gender', 'Education Level']


encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)


In [8]:
scaler = StandardScaler()
num_features = ['Age', 'Years of Experience']
X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])

In [9]:
metrics = regression_cross_validate(KNeighborsRegressor, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_table(*metrics)

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_upd, y_train)

y_pred = knn.predict(X_test_upd)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |        Std Dev |
|:---------|--------------:|---------------:|
| MAE      | 9968.43       | 1442.31        |
| MSE      |    2.2115e+08 |    6.89202e+07 |
| R2       |    0.904675   |    0.0335729   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 206389839.36
Средняя абсолютная ошибка (MAE): 9637.55
Коэффициент детерминации (R^2): 0.91


Можно увидеть что в среднем значения метрик улучшились. Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [10]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        """
        Инициализация KNNRegressor.
        """
        self.n_neighbors = n_neighbors
        self.metric = metric
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
    
    def _calculate_distance(self, x1, x2):
        """
        Вычисляет расстояние между двумя точками в пространстве.
        """
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError(f"Unknown metric: {self.metric}")
    
    def predict(self, X):
        """
        Прогнозирует значения для входных данных.
        """
        X = np.array(X)
        predictions = []
        
        for x in X:
            # Вычисление расстояний до всех точек из тренировочных данных
            distances = np.array([self._calculate_distance(x, x_train) for x_train in self.X_train])
            
            # Находим индексы ближайших соседей
            nearest_neighbors_indices = distances.argsort()[:self.n_neighbors]
            
            # Вычисляем среднее значение целевой переменной ближайших соседей
            nearest_neighbors_values = self.y_train[nearest_neighbors_indices]
            predictions.append(nearest_neighbors_values.mean())
        
        return np.array(predictions)

In [11]:
metrics = regression_cross_validate(KNNRegressor, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_table(*metrics)

knn = KNNRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |       Std Dev |
|:---------|----------------:|--------------:|
| MAE      | 11202           | 587.067       |
| MSE      |     2.89785e+08 |   3.17759e+07 |
| R2       |     0.874733    |   0.0181375   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 300472813.24
Средняя абсолютная ошибка (MAE): 12234.04
Коэффициент детерминации (R^2): 0.86


In [12]:
metrics = regression_cross_validate(KNNRegressor, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_table(*metrics)

knn = KNNRegressor(n_neighbors=5)
knn.fit(X_train_upd, y_train)

y_pred = knn.predict(X_test_upd)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |           Mean |        Std Dev |
|:---------|---------------:|---------------:|
| MAE      | 9793.5         | 1254.12        |
| MSE      |    2.15831e+08 |    5.70089e+07 |
| R2       |    0.907249    |    0.0271058   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 206958827.66
Средняя абсолютная ошибка (MAE): 9615.53
Коэффициент детерминации (R^2): 0.91


Результаты показывают что собственная имплементация модели в среднем работает примерно на том же уровне качества, что и модель из `sklearn`

### Classification

#### 1. Обработка данных

In [13]:
df = pd.read_csv('data/Student_Depression_Dataset.csv')
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [14]:
X, y = df.drop(columns=['Depression', 'id']), df['Depression']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

categorical_features = ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
num_features = ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress', ]

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

X_train[categorical_features] = le.fit_transform(X_train[categorical_features])
X_test[categorical_features] = le.transform(X_test[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

#### 2. Построение бейзлайна 

In [15]:
metrics = classification_cross_validate(KNeighborsClassifier, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_classification_table(*metrics)

knn = KNeighborsClassifier(n_neighbors=3)  # Выбираем количество соседей
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.737539 | 0.00632668 |
| Precision | 0.735315 | 0.00636422 |
| Recall    | 0.737539 | 0.00632668 |
| F1-score  | 0.735266 | 0.00638781 |

=== Результаты на Тесте ===
1. Accuracy: 73.71%
2. Precision: 73.51%
3. Recall: 73.71%
4. F1-score: 73.50%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить число соседей

In [16]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)


scaler = StandardScaler()
X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])

/home/constantfear/.pyenv/versions/3.9.19/envs/labs/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [17]:
metrics = classification_cross_validate(KNeighborsClassifier, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_classification_table(*metrics)

knn = KNeighborsClassifier(n_neighbors=5)  # Выбираем количество соседей
knn.fit(X_train_upd, y_train)

y_pred = knn.predict(X_test_upd)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.814671 | 0.00471789 |
| Precision | 0.814213 | 0.0047123  |
| Recall    | 0.814671 | 0.00471789 |
| F1-score  | 0.813041 | 0.00511674 |

=== Результаты на Тесте ===
1. Accuracy: 80.83%
2. Precision: 80.78%
3. Recall: 80.83%
4. F1-score: 80.67%


Можно увидеть что в среднем значения метрик улучшились. 
Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [18]:
from annoy import AnnoyIndex

In [19]:
class KNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        """
        Инициализация KNNClassifier.
        """
        self.n_neighbors = n_neighbors
        self.n_trees = 10
        self.metric = metric
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.n_features = self.X_train.shape[1]

        # Создаём индекс Annoy
        self.index = AnnoyIndex(self.n_features, metric=self.metric)
        for i, x in enumerate(self.X_train):
            self.index.add_item(i, x)
        
        # Строим деревья
        self.index.build(self.n_trees)
    
    
    def predict(self, X):
        
        """
        Классифицирует входные данные.
        """
        X = np.array(X)
        predictions = []
        
        for x in X:
            # Находим ближайших соседей
            nearest_neighbors_indices = self.index.get_nns_by_vector(x, self.n_neighbors)
            
            # Получаем метки ближайших соседей
            nearest_neighbors_labels = self.y_train[nearest_neighbors_indices]
            
            # Голосование: выбираем наиболее частый класс
            most_common_class = Counter(nearest_neighbors_labels).most_common(1)[0][0]
            predictions.append(most_common_class)
        
        return np.array(predictions)

In [20]:
metrics = classification_cross_validate(KNNClassifier, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_classification_table(*metrics)

knn = KNNClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.729223 | 0.00319656 |
| Precision | 0.726935 | 0.00324859 |
| Recall    | 0.729223 | 0.00319656 |
| F1-score  | 0.727074 | 0.00314616 |

=== Результаты на Тесте ===
1. Accuracy: 72.55%
2. Precision: 72.32%
3. Recall: 72.55%
4. F1-score: 72.31%


In [21]:
metrics = classification_cross_validate(KNNClassifier, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_classification_table(*metrics)

knn = KNNClassifier(n_neighbors=5)
knn.fit(X_train_upd, y_train)

y_pred = knn.predict(X_test_upd)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.809319 | 0.00791926 |
| Precision | 0.808459 | 0.00795994 |
| Recall    | 0.809319 | 0.00791926 |
| F1-score  | 0.807981 | 0.00803129 |

=== Результаты на Тесте ===
1. Accuracy: 79.97%
2. Precision: 79.88%
3. Recall: 79.97%
4. F1-score: 79.87%


Результаты показывают что собственная имплементация модели в среднем работает примерно на том же уровне качества, что и модель из `sklearn`

### Заключение

Внесённые изменения, включая нормализацию данных и увеличение числа соседей в алгоритме поиска ближайших соседей, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 2.59e+08  | 11612.23   |  0.88      |
| Sklearn (после улучшения) | 2.063e+08 | 9637.55    |  0.91      |
| Собственная имплементация (до улучшения)   | 3.004e+08 | 12234.04   |  0.86      |
| Собственная имплементация (после улучшения)| 2.069e+08  | 9615.53    |  0.91      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-scor |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   73.71%  |   73.51%   |  73.71%    |  73.50%    |
| Sklearn (после улучшения) |   80.83%  |   80.78%   |  80.83%    |  80.67%    |
| Собственная имплементация (до улучшения)   |   72.55%  |   72.32%   |  72.55%    |  72.31%    |
| Собственная имплементация (после улучшения)|   79.97%  |   79.88%   |  79.97%    |  79.87%    |